In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats

import matplotlib.pyplot as plt
import seaborn as sns
import env
import acquire
import explore
import prepare

In [2]:
#Sets options to show more information
pd.options.display.max_columns = None
pd.options.display.width = 100
pd.options.display.max_colwidth = None
pd.options.display.max_rows = 200

In [3]:
df = acquire.wrangle_zillow()

In [4]:
df_sorted = df.sort_values(['parcelid','transactiondate'], ascending=False)

In [5]:
df_sorted = df_sorted[~df_sorted['parcelid'].duplicated()]

In [6]:
#Fill with zeros
cols_to_fill = ['basementsqft', 'decktypeid', 'fireplacecnt', 'garagecarcnt', 'garagetotalsqft',
                'hashottuborspa', 'poolcnt', 'poolsizesum', 'pooltypeid2','pooltypeid7',
                'pooltypeid10','fireplaceflag', 'taxdelinquencyflag', 'airconditioningdesc'] 

In [7]:
df_sorted[cols_to_fill] = df_sorted[cols_to_fill].fillna(0)

In [8]:
df_sorted = df_sorted[df_sorted['latitude'].notna()]

In [9]:
df_sorted.head()

,parcelid,typeconstructiontypeid,storytypeid,propertylandusetypeid,heatingorsystemtypeid,buildingclasstypeid,architecturalstyletypeid,airconditioningtypeid,id,basementsqft,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,decktypeid,finishedfloor1squarefeet,calculatedfinishedsquarefeet,finishedsquarefeet12,finishedsquarefeet13,finishedsquarefeet15,finishedsquarefeet50,finishedsquarefeet6,fips,fireplacecnt,fullbathcnt,garagecarcnt,garagetotalsqft,hashottuborspa,latitude,longitude,lotsizesquarefeet,poolcnt,poolsizesum,pooltypeid10,pooltypeid2,pooltypeid7,propertycountylandusecode,propertyzoningdesc,rawcensustractandblock,regionidcity,regionidcounty,regionidneighborhood,regionidzip,roomcnt,threequarterbathnbr,unitcnt,yardbuildingsqft17,yardbuildingsqft26,yearbuilt,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,id.1,logerror,transactiondate,propertylandusedesc,storydesc,typeconstructiondesc
16654,167689317,NaN,NaN,269.0,NaN,NaN,NaN,NaN,2535331,0.0,0.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6037.0,0.0,NaN,0.0,0.0,0.0,34045062.0,-118020570.0,NaN,0.0,0.0,0.0,0.0,0.0,010D,EMR1B*,6.037434e+07,NaN,3101.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,318400.0,437387.0,2016.0,118987.0,5947.25,0,NaN,NaN,0,NaN,NaN,NaN,16654,-0.008789,2017-03-14,Planned Unit Development,NaN,NaN
7892,167688532,NaN,NaN,266.0,2.0,NaN,NaN,1.0,2758757,0.0,3.0,3.0,4.0,3.0,0.0,NaN,1661.0,1661.0,NaN,NaN,NaN,NaN,6037.0,0.0,3.0,0.0,0.0,0.0,34108983.0,-118262402.0,NaN,0.0,0.0,0.0,0.0,0.0,010C,LARD2,6.037187e+07,NaN,3101.0,NaN,NaN,0.0,NaN,1.0,NaN,NaN,2016.0,NaN,0.0,NaN,147921.0,2016.0,147921.0,1902.75,0,NaN,NaN,Central,NaN,NaN,Central,7892,0.006706,2017-02-03,Condominium,NaN,NaN
41688,167687839,NaN,NaN,261.0,NaN,NaN,NaN,NaN,1372384,0.0,0.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6037.0,0.0,NaN,0.0,0.0,0.0,34048223.0,-118520239.0,NaN,0.0,0.0,0.0,0.0,0.0,0100,LAR1,6.037263e+07,NaN,3101.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,1842678.0,2016.0,1842678.0,22045.81,0,NaN,NaN,0,NaN,NaN,NaN,41688,0.038797,2017-05-31,Single Family Residential,NaN,NaN
14557,167687739,NaN,NaN,266.0,NaN,NaN,NaN,NaN,2863262,0.0,0.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6037.0,0.0,NaN,0.0,0.0,0.0,34041716.0,-118455310.0,NaN,0.0,0.0,0.0,0.0,0.0,010C,LAR3,6.037268e+07,NaN,3101.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,5451600.0,6440197.0,2016.0,988597.0,77045.13,0,NaN,NaN,0,NaN,NaN,NaN,14557,0.360020,2017-03-03,Condominium,NaN,NaN
13467,167686999,NaN,NaN,261.0,NaN,NaN,NaN,NaN,775695,0.0,0.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6037.0,0.0,NaN,0.0,0.0,0.0,34424104.0,-118468083.0,NaN,0.0,0.0,0.0,0.0,0.0,0100,SCRM,6.037920e+07,NaN,3101.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,26405.0,2016.0,26405.0,988.48,0,NaN,NaN,0,NaN,NaN,NaN,13467,-0.068632,2017-02-28,Single Family Residential,NaN,NaN


In [10]:
df_sorted = prepare.handle_missing_values(df_sorted)

Size of Correlation	Interpretation

|.90 to 1.00| (−.90 to −1.00)|	Very high positive (negative) correlation|

|.70 to .90| (−.70 to −.90)|	High positive (negative) correlation|

|.50 to .70| (−.50 to −.70)|	Moderate positive (negative) correlation|

|.30 to .50| (−.30 to −.50)|	Low positive (negative) correlation|

|.00 to .30| (.00 to −.30)|	negligible correlation|

In [11]:
df_sorted.shape

(77381, 50)

In [ ]:
#function input:
df
model = "" #Unsure how to handle this.
percent_to_trim = .25
trim_upper = True
trim_lower = True
